# Pop2Piano - Inference (Use Ready Model) 🎹

This notebook is for generating piano covers using the pre-trained Pop2Piano model.

## Key Steps:
1. **Setup Environment:** Install dependencies and fix compatibility issues.
2. **Clone Repository:** Get the code.
3. **Load Model:** Download/Load the pre-trained model.
4. **Generate:** Upload audio and create piano MIDI.

In [ ]:
# @title 1. Mount Google Drive 💾
from google.colab import drive
import os

drive.mount('/content/drive')

# Create a persistent working directory in your Google Drive
GDRIVE_WORKING_DIR = "/content/drive/MyDrive/Pop2Piano_Workspace"
if not os.path.exists(GDRIVE_WORKING_DIR):
    os.makedirs(GDRIVE_WORKING_DIR)

# Change to the persistent directory
%cd {GDRIVE_WORKING_DIR}

print(f"Switched to persistent directory: {os.getcwd()}")

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/huggingface/transformers.git"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "librosa", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Apply Magic Fixes (Essentia Mocking) 🪄
import sys
from unittest.mock import MagicMock
from importlib.machinery import ModuleSpec

# Fix for missing essentia
mock_essentia = MagicMock()
# Important fix for Python 3.12+ (spec must be set)
mock_essentia.__spec__ = ModuleSpec(name='essentia', loader=None)

sys.modules["essentia"] = mock_essentia
sys.modules["essentia.standard"] = MagicMock()

print("Essentia mocked successfully! 🧙‍♂️")

In [ ]:
# @title 3. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git pull
else:
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
    %cd {repo_name}

print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title 4. Load Pre-trained Model & Generate 🎹
from transformers import Pop2PianoForConditionalGeneration, AutoProcessor
import librosa
import soundfile as sf
import torch

print("⏳ Loading Model...")
model = Pop2PianoForConditionalGeneration.from_pretrained("sweetcocoa/pop2piano")
processor = AutoProcessor.from_pretrained("sweetcocoa/pop2piano")
print("✅ Model Loaded!")

# Example Usage (Uncomment to use):
# audio, rate = librosa.load("your_audio_file.wav")
# inputs = processor(audio=audio, sampling_rate=rate, return_tensors="pt")
# outputs = model.generate(**inputs, composer="composer_name")
# processor.batch_decode(outputs, save_path="output_midi.mid")